# Demo Covid-19 Timeline in Thammasat University

โดยจะเก็บข้อมูล ผู้เสี่ยงติดโรคทุกระดับ (PUI, High Risk, Low Risk, Healthy) และสถานที่ที่กลุ่มคนเหล่านี้เคยไป รวมถึงวันเวลาด้วย

### Technical Information
ใช้ Neo4j ในการเก็บข้อมูลแบบ Graph Database (run `docker-compose up`) แล้วใช้ Python ดึงข้อมูลจาก Neo4j โดยใช้ Py2neo driver แล้วแสดงผลเป็นกราฟด้วย vis.js (file html อยู่ใน `./figure/`)

## Insert Mock Data


In [58]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='1234')

In [59]:
graph.delete_all()

In [60]:
from py2neo import Node
yo = Node("Person", name="โย", level='PUI')
mod = Node("Person", name="มด", level='HighRisk')
pong = Node("Person", name="ป้อง", level='HighRisk')
pan = Node("Person", name="แป้น", level='LowRisk')
am = Node("Person", name="แอม", level='Healthy')
num = Node("Person", name="น้ำ", level='Healthy')
day = Node("Person", name="เดย์", level='LowRisk')
mui = Node("Person", name="มุ้ย", level='HighRisk')
ann = Node("Person", name="แอน", level='LowRisk')
tong = Node("Person", name="โต้ง", level='Healthy')

tu = Node("Location", name="TU")
lfc = Node("Location", name="Lazy Fat Cat")
ai = Node("Location", name="AI Lab")
cath = Node("Location", name="Kitchen Cath")
airport = Node("Location", name="Airport")

graph.create(yo | mod | pong | pan | am | num | day | mui | ann | tong | tu | lfc | ai | cath | airport)

In [61]:
from py2neo import Relationship
from datetime import datetime

graph.create(Relationship(yo, "WENT_TO", tu, start=str(datetime(2020, 3, 17)), end=str(datetime(2020, 3, 19))))
graph.create(Relationship(mod, "WENT_TO", tu, start=str(datetime(2020, 3, 16)), end=str(datetime(2020, 3, 19))))
graph.create(Relationship(pong, "WENT_TO", tu, start=str(datetime(2020, 3, 17)), end=str(datetime(2020, 3, 18))))
graph.create(Relationship(num, "WENT_TO", tu, start=str(datetime(2020, 3, 10)), end=str(datetime(2020, 3, 19))))
graph.create(Relationship(am, "WENT_TO", tu, start=str(datetime(2020, 3, 9)), end=str(datetime(2020, 3, 18))))
graph.create(Relationship(pan, "WENT_TO", lfc, start=str(datetime(2020, 3, 1)), end=str(datetime(2020, 3, 5))))
graph.create(Relationship(pan, "WENT_TO", ai, start=str(datetime(2020, 3, 2)), end=str(datetime(2020, 3, 8))))
graph.create(Relationship(pong, "WENT_TO", ai, start=str(datetime(2020, 3, 3)), end=str(datetime(2020, 3, 5))))
graph.create(Relationship(mod, "WENT_TO", cath, start=str(datetime(2020, 3, 7)), end=str(datetime(2020, 3, 10))))
graph.create(Relationship(pan, "WENT_TO", cath, start=str(datetime(2020, 3, 9)), end=str(datetime(2020, 3, 10))))
graph.create(Relationship(day, "WENT_TO", lfc, start=str(datetime(2020, 3, 8)), end=str(datetime(2020, 3, 10))))
graph.create(Relationship(tong, "WENT_TO", airport, start=str(datetime(2020, 2, 21)), end=str(datetime(2020, 2, 23))))
graph.create(Relationship(ann, "WENT_TO", airport, start=str(datetime(2020, 2, 21)), end=str(datetime(2020, 2, 23))))

graph.create(Relationship(pong, "CLOSE_FRIEND", mod))
graph.create(Relationship(mod, "CLOSE_FRIEND", pong))
graph.create(Relationship(yo, "CLOSE_FRIEND", mui))
graph.create(Relationship(mui, "CLOSE_FRIEND", yo))


## Show Graph Data

In [62]:
from scripts.vis import draw, vis_network

options = {"Person": "name", "Drink": "name", "Manufacturer": "name"}
# draw(graph, options)

In [63]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='1234')

In [64]:
query = """
MATCH (p:Person)
OPTIONAL MATCH (p)-[a]->(m)
RETURN  p,
        id(p) as p_id,
        a,
        m,
        id(m) as m_id;
"""

data = graph.run(query)
data

In [65]:
def get_node_info(node, node_id):
    gr = ''.join(str(node.labels).split(':'))
    return {"id": node_id, "label": node['name'], "group": gr, "level": node['level']}

In [66]:
data = graph.run(query)

nodes = []
edges = []
physics = True

for row in data:
    p = row[0]
    p_id = row[1]
    a = row[2]
    m = row[3]
    m_id = row[4]
    
    p_info = get_node_info(p, p_id)
    if p_info not in nodes:
        nodes.append(p_info)
         
    if m:
        m_info = get_node_info(m, m_id)
        if m_info not in nodes:
            nodes.append(m_info)
        
        if type(a).__name__ == 'WENT_TO':
            edges.append({"from": p_info["id"], "to": m_info["id"], "label": type(a).__name__, "start":a["start"], "end":a["end"]})
        else:
            edges.append({"from": p_info["id"], "to": m_info["id"], "label": type(a).__name__})

In [67]:
import json
with open('aa.json', 'w') as outfile:
    json.dump({'nodes':nodes, 'edges':edges}, outfile)